In [1]:
import pandas as pd 
import numpy as np

In [2]:
#evidence count for disease_gene association
path = '/Users/f.nasirian/Dropbox (CCNR)/Biology/99_Toolbox/data/Open_Targets_Platform/data/out'
dgs = pd.read_csv(path+'/disease_gene_v2.csv').drop('targets', axis=1).drop_duplicates()
dgs = dgs[dgs.GeneId.notnull()]
dgs = dgs[(~dgs.source.isin(['literature']))]
#dgs = dgs.groupby(['efoId', 'efoName', 'GeneId', 'source'])['evidenceCount'].apply(sum).reset_index()

In [3]:
dgs.source.unique()

array(['genetic_association', 'known_drug', 'somatic_mutation',
       'rna_expression', 'affected_pathway', 'animal_model'], dtype=object)

In [4]:
#disease therapeutic area
ta = dgs[['efoId']].drop_duplicates().merge(
    pd.read_csv(path+'/therapeutic_areas_v2.csv').rename(columns={'efoName':'taName'}), on='efoId', how='left')    

In [5]:
#diseases classified in cvd area
cvd_area = set(ta[ta.taName == 'cardiovascular disease']['efoId']) - {'EFO_0000319'}

In [6]:
dgs = dgs[(dgs.efoId.isin(cvd_area)) & (dgs.source == 'genetic_association')]

In [7]:
#cvds with at least 20 associations
df = dgs.groupby(['efoId', 'efoName'])['GeneId'].size().reset_index(name='Count')
sel_cvd = df[df.Count > 20].efoId.unique()
len(sel_cvd)

65

In [8]:
dgs[(dgs.efoId.isin(sel_cvd)) & (dgs.score > 0.1)].groupby(['efoId', 'efoName'])['GeneId'].size().sort_values()

efoId          efoName                            
EFO_1001976    cardioembolic stroke                    13
MONDO_0024664  hypertension, pregnancy-induced         15
EFO_0003875    peripheral vascular disease             15
EFO_0005938    congenital left-sided heart lesions     15
EFO_0003914    atherosclerosis                         16
                                                     ... 
EFO_0000712    stroke                                 152
EFO_0000612    myocardial infarction                  224
EFO_0000275    atrial fibrillation                    348
EFO_0001645    coronary artery disease                571
EFO_0000537    hypertension                           576
Name: GeneId, Length: 65, dtype: int64

In [9]:
dgs[(dgs.efoId.isin(sel_cvd)) & (dgs.score > 0.1)].to_csv('data/cvd_genes.csv', index=False)